# Calculate OD

In [2]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

# pip install osmium
# import osmium, logging
# import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

In [3]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [4]:
from GOSTnets.load_traffic2 import *

In [5]:
#read_nodes = pd.read_csv('./vavuniya_unclean_nodes.csv')

In [6]:
#read_nodes[:5]

In [7]:
# read graph
G = nx.read_gpickle('./sri_lanka_unclean2_w_time_largest.pickle')

In [8]:
len(G.edges)

1675316

In [9]:
gn.example_edge(G, 5)

(1650104033, 6236632580, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f64f9e97e10>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 9.757789924575977, 'speed': 26.793650793650794, 'time': 1.3110585040840237, 'mode': 'drive'})
(1650104033, 970058024, {'osm_id': 152177791, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f64f9e9c110>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 16.0020254611234, 'speed': 26.793650793650794, 'time': 2.150035174515869, 'mode': 'drive'})
(6236632580, 6236632579, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f64f9e9c1d0>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 5.235256075996137, 'speed': 26.793650793650794, 'time': 0.7034099988364477, 'mode': 'drive'})
(6236632580, 1650104033, {'osm_i

## load origins

In [10]:
origins = gpd.read_file('./origins_destinations/cities_top10.shp')

In [11]:
origins

,osm_id,code,fclass,population,name,country,geometry
0,44240758,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76707 6.97382)
1,50794342,1005,national_capital,753000,Colombo,LKA,POINT (79.85385 6.93500)
2,258049423,1001,city,40800,Matale,LKA,POINT (80.62342 7.47213)
3,330173100,1001,city,45400,Puttalam,LKA,POINT (79.82866 8.03019)
4,335253618,1001,city,99100,Trincomalee,LKA,POINT (81.23450 8.57643)
5,2050947471,1001,city,600000,Jaffna,LKA,POINT (80.00930 9.66509)
6,2908772560,1001,city,125000,Kandy,LKA,POINT (80.63508 7.29309)
7,3289447451,1001,city,20000,Gampaha,LKA,POINT (79.99514 7.09256)
8,3639539242,1001,city,110000,Matara,LKA,POINT (80.54829 5.94782)
9,3639546887,1001,city,120000,Galle,LKA,POINT (80.21495 6.03281)


## load destinations

In [12]:
destinations = gpd.read_file('./origins_destinations/tourism_w_cities_wgs84.shp')

In [13]:
destinations

,Loc_name,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,StAddr,Block,Sector,...,PostalExt,Country,LangCode,NEAR_FID,NEAR_DIST,NEAR_X,NEAR_Y,nearCtyPop,name,geometry
0,World,"Embekka Katharagama Dewalaya, Embekka, LKA",Embekka Katharagama Dewalaya,POI,Temple,Embekka Katharagama Dewalaya,Embekka,None,None,None,...,None,LKA,ENG,7,11110.595704,459719.265399,806165.680488,125000,Kandy,POINT (80.56806 7.21811)
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,3,15374.643812,458448.971316,825958.949979,40800,Matale,POINT (80.74294 7.40064)
2,World,"Royal Palace Park-Kandy, Keerthie Sri Rajasing...",Royal Palace Park-Kandy,POI,Park,Royal Palace Park-Kandy,"Keerthie Sri Rajasinghe Mawatha, Kandy",Keerthie Sri Rajasinghe Mawatha,None,None,...,None,LKA,ENG,7,482.167726,459719.265399,806165.680488,125000,Kandy,POINT (80.63751 7.28947)
3,World,"Kandy, LKA",Kandy,Locality,City,Kandy,None,None,None,None,...,None,LKA,ENG,7,446.690670,459719.265399,806165.680488,125000,Kandy,POINT (80.63866 7.29497)
4,World,"Bauddha Bala Mandalaya, Akkara Road, Ranawana ...",Bauddha Bala Mandalaya,POI,Temple,Bauddha Bala Mandalaya,"Akkara Road, Ranawana East",Akkara Road,None,None,...,None,LKA,ENG,7,4130.237703,459719.265399,806165.680488,125000,Kandy,POINT (80.62270 7.32835)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,World,"Wasana Trade Point, Temple Road, Negombo, LKA",Wasana Trade Point,POI,Business Facility,Wasana Trade Point,"Temple Road, Negombo",Temple Road,None,None,...,None,LKA,ENG,8,18946.453259,389028.597467,784101.180029,20000,Gampaha,POINT (79.85915 7.19703)
410,World,"Post Office-Wadduwa, Wadduwa Station Road, Wad...",Post Office-Wadduwa,POI,Post Office,Post Office-Wadduwa,"Wadduwa Station Road, Wadduwa South",Wadduwa Station Road,None,None,...,None,LKA,ENG,2,31162.749630,373380.454933,766715.939941,753000,Colombo,POINT (79.92945 6.66347)
411,World,"Madusanka, A8, Wekada West, LKA",Madusanka,POI,Grocery,Madusanka,"A8, Wekada West",A8,None,None,...,None,LKA,ENG,2,25714.185503,373380.454933,766715.939941,753000,Colombo,POINT (79.91315 6.71011)
412,World,"Post Office-Beruwala, Mangala Road, Paranakade...",Post Office-Beruwala,POI,Post Office,Post Office-Beruwala,"Mangala Road, Paranakade",Mangala Road,None,None,...,None,LKA,ENG,2,53062.770683,373380.454933,766715.939941,753000,Colombo,POINT (79.98347 6.47290)


In [14]:
origins_gdf = gn.pandana_snap_c(G, origins, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [15]:
origins_gdf

,osm_id,code,fclass,population,name,country,geometry,NN,NN_dist
0,44240758,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76707 6.97382),4784044133,1.631987
1,50794342,1005,national_capital,753000,Colombo,LKA,POINT (79.85385 6.93500),5385270047,6.505032
2,258049423,1001,city,40800,Matale,LKA,POINT (80.62342 7.47213),1727494683,82.604133
3,330173100,1001,city,45400,Puttalam,LKA,POINT (79.82866 8.03019),4140985760,7.214712
4,335253618,1001,city,99100,Trincomalee,LKA,POINT (81.23450 8.57643),1243386867,7.611852
5,2050947471,1001,city,600000,Jaffna,LKA,POINT (80.00930 9.66509),7098642100,6.705113
6,2908772560,1001,city,125000,Kandy,LKA,POINT (80.63508 7.29309),4004620073,11.021930
7,3289447451,1001,city,20000,Gampaha,LKA,POINT (79.99514 7.09256),3289453904,11.707473
8,3639539242,1001,city,110000,Matara,LKA,POINT (80.54829 5.94782),1734550181,10.113321
9,3639546887,1001,city,120000,Galle,LKA,POINT (80.21495 6.03281),3379679501,8.177613


In [16]:
origins_list = list(set(origins_gdf.NN))

In [17]:
destinations_gdf = gn.pandana_snap_c(G, destinations, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [18]:
destinations_gdf

,Loc_name,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,StAddr,Block,Sector,...,LangCode,NEAR_FID,NEAR_DIST,NEAR_X,NEAR_Y,nearCtyPop,name,geometry,NN,NN_dist
0,World,"Embekka Katharagama Dewalaya, Embekka, LKA",Embekka Katharagama Dewalaya,POI,Temple,Embekka Katharagama Dewalaya,Embekka,None,None,None,...,ENG,7,11110.595704,459719.265399,806165.680488,125000,Kandy,POINT (80.56806 7.21811),1743126554,14.430923
1,None,None,None,None,None,None,None,None,None,None,...,None,3,15374.643812,458448.971316,825958.949979,40800,Matale,POINT (80.74294 7.40064),3645433572,120.831359
2,World,"Royal Palace Park-Kandy, Keerthie Sri Rajasing...",Royal Palace Park-Kandy,POI,Park,Royal Palace Park-Kandy,"Keerthie Sri Rajasinghe Mawatha, Kandy",Keerthie Sri Rajasinghe Mawatha,None,None,...,ENG,7,482.167726,459719.265399,806165.680488,125000,Kandy,POINT (80.63751 7.28947),3804266901,7.492919
3,World,"Kandy, LKA",Kandy,Locality,City,Kandy,None,None,None,None,...,ENG,7,446.690670,459719.265399,806165.680488,125000,Kandy,POINT (80.63866 7.29497),5507321512,29.207214
4,World,"Bauddha Bala Mandalaya, Akkara Road, Ranawana ...",Bauddha Bala Mandalaya,POI,Temple,Bauddha Bala Mandalaya,"Akkara Road, Ranawana East",Akkara Road,None,None,...,ENG,7,4130.237703,459719.265399,806165.680488,125000,Kandy,POINT (80.62270 7.32835),2342916291,110.833504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,World,"Wasana Trade Point, Temple Road, Negombo, LKA",Wasana Trade Point,POI,Business Facility,Wasana Trade Point,"Temple Road, Negombo",Temple Road,None,None,...,ENG,8,18946.453259,389028.597467,784101.180029,20000,Gampaha,POINT (79.85915 7.19703),3804238095,24.739240
410,World,"Post Office-Wadduwa, Wadduwa Station Road, Wad...",Post Office-Wadduwa,POI,Post Office,Post Office-Wadduwa,"Wadduwa Station Road, Wadduwa South",Wadduwa Station Road,None,None,...,ENG,2,31162.749630,373380.454933,766715.939941,753000,Colombo,POINT (79.92945 6.66347),5397260742,0.948475
411,World,"Madusanka, A8, Wekada West, LKA",Madusanka,POI,Grocery,Madusanka,"A8, Wekada West",A8,None,None,...,ENG,2,25714.185503,373380.454933,766715.939941,753000,Colombo,POINT (79.91315 6.71011),1446610115,4.806681
412,World,"Post Office-Beruwala, Mangala Road, Paranakade...",Post Office-Beruwala,POI,Post Office,Post Office-Beruwala,"Mangala Road, Paranakade",Mangala Road,None,None,...,ENG,2,53062.770683,373380.454933,766715.939941,753000,Colombo,POINT (79.98347 6.47290),1606696560,91.068186


In [19]:
destinations_list = list(set(destinations_gdf.NN))

In [20]:
destinations_list

[2291208197,
 1667080206,
 6991570964,
 6117715989,
 1743126554,
 2100457519,
 1185558575,
 130445367,
 2085208123,
 4047450172,
 1242787912,
 1691609164,
 4891658323,
 4893239380,
 1517451347,
 3799898203,
 2094557278,
 7066196064,
 3587975265,
 2078574693,
 1745895527,
 1741201515,
 3249014896,
 4784988278,
 1895649399,
 3208652929,
 610939019,
 3800084623,
 666892434,
 1242787995,
 308865188,
 3806787750,
 4248172715,
 47823021,
 1146790064,
 5580828849,
 4600967349,
 3902560439,
 3081279676,
 5497589950,
 2342916291,
 3985412306,
 1709670615,
 1722822882,
 3851290872,
 6988599546,
 1924788477,
 7083442431,
 61802753,
 5596123398,
 4111669517,
 3804238095,
 2833357077,
 2634987798,
 5504295192,
 5997193508,
 3686420776,
 4579338537,
 4007776558,
 3803965753,
 6181355836,
 1269985600,
 2145161538,
 3213648201,
 2079873354,
 1952715084,
 849219917,
 4217581902,
 2151514445,
 5265641820,
 2902126951,
 5874008423,
 1974698347,
 3169970562,
 1896380803,
 5216672133,
 7350159754,
 2087539

## Calculate OD

In [21]:
# It will use the default weight of 'time'
import time

start = time.time()

OD = gn.calculate_OD(G, origins_list, destinations_list, fail_value = 9999999)

end = time.time()
print(end - start)

90.5690667629242


In [22]:
OD

array([[18510.07439113, 12507.29594792, 10194.32685517, ...,
         4829.20915756, 18447.90111899, 10138.05719571],
       [18168.79048334, 16420.96031958, 15970.77124844, ...,
        17919.4150786 , 17941.59963555, 15914.50158898],
       [22253.89341784,  5577.87119714,  7919.8878946 , ...,
        19234.27084502, 22026.70257005,  8107.09245389],
       ...,
       [23877.89053379, 28003.1856312 , 26374.80525088, ...,
        12465.60768133, 23815.71726166, 26318.53559141],
       [13600.79878917, 13830.61934095, 12441.78656499, ...,
         8611.52067083, 13538.62551703, 12385.51690552],
       [22760.36981733,  4335.42794116,   527.0371602 , ...,
        13976.55410784, 22698.19654519,   257.35757222]])

In [23]:
OD_df = pd.DataFrame(OD, index = origins_list, columns = destinations_list)

In [24]:
OD_df

,2291208197,1667080206,6991570964,6117715989,1743126554,2100457519,1185558575,130445367,2085208123,4047450172,...,3772792789,3772792790,1730404311,258523099,6564888541,3216934896,1506164725,1657419767,1675585534,4621961215
4140985760,18510.074391,12507.295948,10194.326855,17017.046783,11058.179415,14588.843982,11616.506298,14153.017294,21992.550175,9847.193878,...,10785.029235,10774.574424,20191.061674,11498.594777,12598.994605,12013.650506,15059.136708,4829.209158,18447.901119,10138.057196
4784044133,18168.790483,16420.960320,15970.771248,246.884367,8132.610523,26436.198542,16370.683271,18429.611369,10032.738931,7421.909044,...,16542.592663,16551.018817,12617.043637,15568.322303,24446.349165,9460.677768,17173.575547,17919.415079,17941.599636,15914.501589
1734550181,22253.893418,5577.871197,7919.887895,15703.435161,15053.684958,28808.955754,6628.789813,4084.405129,12451.921281,15797.568395,...,7155.992360,7166.447171,5608.141428,20767.059336,26819.106377,11771.937683,2828.369308,19234.270845,22026.702570,8107.092454
4004620073,15357.120897,12269.390354,10880.557578,7577.372332,2363.735176,19278.819754,11542.389592,13915.111700,16917.251506,175.355078,...,11471.259958,11460.805147,17386.911484,9107.722890,17288.970377,7586.305618,14821.231114,11054.161876,15129.930049,10824.287919
3379679501,23632.568454,4604.393388,6946.410085,17082.110198,14080.207149,27835.477944,5655.312004,2641.929211,13830.596317,14824.090585,...,6182.514551,6192.969362,6986.816464,19793.581527,25845.628567,10798.459874,248.574822,18260.793036,23405.377606,7133.614644
3289453904,20834.449315,4373.582498,2984.749722,13897.581977,8109.595324,21754.640756,3646.581736,6019.303844,14196.753964,8853.478760,...,3575.452102,3564.997291,12057.348224,13822.969702,19764.791379,5474.782889,6925.423258,12179.955848,20772.276043,2928.480063
1243386867,10050.216305,21028.939802,19640.107026,21176.982762,15603.274375,7627.850486,20301.939039,22674.661147,25899.034561,13597.703797,...,20230.809405,20220.354595,24473.158962,6771.671943,10369.806621,17918.607278,23580.780562,7247.822892,9988.043033,19583.837366
7098642100,23877.890534,28003.185631,26374.805251,29229.796481,23656.088094,9325.359153,27276.184869,29648.906977,34941.923384,21650.517516,...,26965.507630,26955.052820,33516.047784,17718.037277,5435.996178,25167.001175,30555.026391,12465.607681,23815.717262,26318.535591
1727494683,13600.798789,13830.619341,12441.786565,9922.327601,4401.428912,16760.416039,13103.618578,15476.340687,19262.206776,2343.048636,...,13032.488944,13022.034134,18380.281612,6589.319175,14770.566663,9819.827132,16382.460101,8611.520671,13538.625517,12385.516906
5385270047,22760.369817,4335.427941,527.037160,15823.502479,10035.515826,23551.239017,2070.903425,6179.747102,15114.659932,10779.399262,...,1389.961431,1379.506621,12217.791482,15748.890204,21561.389640,7361.264357,7085.866516,13976.554108,22698.196545,257.357572


In [25]:
#OD_df.min(axis=0)

2291208197    10050.216305
1667080206     4335.427941
6991570964      527.037160
6117715989      246.884367
1743126554     2363.735176
                  ...     
3216934896     5474.782889
1506164725      248.574822
1657419767     4829.209158
1675585534     9988.043033
4621961215      257.357572
Length: 337, dtype: float64

In [26]:
# first column is the destination (tourist point), and the second column is the nearest city
OD_df.idxmin(axis=0)

2291208197    1243386867
1667080206    5385270047
6991570964    5385270047
6117715989    4784044133
1743126554    4004620073
                 ...    
3216934896    3289453904
1506164725    3379679501
1657419767    4140985760
1675585534    1243386867
4621961215    5385270047
Length: 337, dtype: int64

In [27]:
#OD_df.idxmin(axis=0).to_frame(0)

,0
2291208197,1243386867
1667080206,5385270047
6991570964,5385270047
6117715989,4784044133
1743126554,4004620073
...,...
3216934896,3289453904
1506164725,3379679501
1657419767,4140985760
1675585534,1243386867


In [76]:
# takes the min index value of each column, then groups by the first index (0) and takes the first entry
groupby_obj = OD_df.idxmin(axis=0).to_frame(0).groupby(0)[0]

In [74]:
groupby_obj

In [43]:
groupby_obj.apply(list)

0
1243386867    [1243386867, 1243386867, 1243386867, 124338686...
1727494683    [1727494683, 1727494683, 1727494683, 172749468...
1734550181    [1734550181, 1734550181, 1734550181, 173455018...
3289453904    [3289453904, 3289453904, 3289453904, 328945390...
3379679501    [3379679501, 3379679501, 3379679501, 337967950...
4004620073    [4004620073, 4004620073, 4004620073, 400462007...
4140985760    [4140985760, 4140985760, 4140985760, 414098576...
4784044133    [4784044133, 4784044133, 4784044133, 478404413...
5385270047    [5385270047, 5385270047, 5385270047, 538527004...
7098642100    [7098642100, 7098642100, 7098642100, 709864210...
Name: 0, dtype: object

In [58]:
type(groupby_obj.apply(list))

pandas.core.series.Series

In [71]:
dict_ex = groupby_obj.apply(list).to_dict()

In [73]:
dict_ex

{1243386867: [1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867,
  1243386867],
 1727494683: [1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494683,
  1727494

In [117]:
city_tourist_pt_dict = {}
for name, group in groupby_obj:
    #print(group)
    for items in group.iteritems(): 
        print(items[1])
        if items[1] not in city_tourist_pt_dict:
            city_tourist_pt_dict[items[1]] = [items[0]]
        else:
            #append value to list in dict value
            city_tourist_pt_dict[items[1]].append(items[0])
    #print(type(group))
    #print(group.head(1))

    #print(name)
    #print(city_tourist_pt_dict[group])

1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1243386867
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1727494683
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181
1734550181

In [118]:
city_tourist_pt_dict

{1243386867: [2291208197,
  2100457519,
  1242787912,
  1895649399,
  1242787995,
  3806787750,
  4248172715,
  5580828849,
  3851290872,
  4579338537,
  1896380803,
  1884656092,
  1896380948,
  1880627748,
  6177931854,
  7076053840,
  3985464364,
  4542639194,
  4376980578,
  3805734132,
  3315891502,
  2278210866,
  3893372372,
  7025200613,
  2341660185,
  6021684779,
  1676588668,
  3350242971,
  1685802748,
  1723379544,
  2477387622,
  1775835028,
  3165939618,
  5965445027,
  1675585459,
  1675585534],
 1727494683: [4784988278,
  3208652929,
  6988599546,
  2833357077,
  6181355836,
  3213648201,
  2646372818,
  5441882639,
  5035668064,
  2051261052,
  2279244515,
  1692965634,
  2831541077,
  3748668362,
  3218396149,
  1241535503,
  4924478499,
  1241527387,
  5644231795,
  1241527444,
  258057398,
  3280149722,
  1241527517,
  2833347872,
  2821176699,
  5483879958,
  2297833003,
  3218405040,
  1241534200,
  2303446810,
  3999442747,
  2799732582,
  2799732584,
  34724535

In [109]:
groupby_obj.describe()

,count,mean,std,min,25%,50%,75%,max
0,,,,,,,,
1243386867,36.0,1.243387e+09,0.0,1.243387e+09,1.243387e+09,1.243387e+09,1.243387e+09,1.243387e+09
1727494683,35.0,1.727495e+09,0.0,1.727495e+09,1.727495e+09,1.727495e+09,1.727495e+09,1.727495e+09
1734550181,39.0,1.734550e+09,0.0,1.734550e+09,1.734550e+09,1.734550e+09,1.734550e+09,1.734550e+09
3289453904,15.0,3.289454e+09,0.0,3.289454e+09,3.289454e+09,3.289454e+09,3.289454e+09,3.289454e+09
3379679501,29.0,3.379680e+09,0.0,3.379680e+09,3.379680e+09,3.379680e+09,3.379680e+09,3.379680e+09
4004620073,43.0,4.004620e+09,0.0,4.004620e+09,4.004620e+09,4.004620e+09,4.004620e+09,4.004620e+09
4140985760,36.0,4.140986e+09,0.0,4.140986e+09,4.140986e+09,4.140986e+09,4.140986e+09,4.140986e+09
4784044133,35.0,4.784044e+09,0.0,4.784044e+09,4.784044e+09,4.784044e+09,4.784044e+09,4.784044e+09
5385270047,51.0,5.385270e+09,0.0,5.385270e+09,5.385270e+09,5.385270e+09,5.385270e+09,5.385270e+09


## Loop through dictionary in order to do a calculate_OD for each city's nearest tourist sites

In [119]:
OD = {}
OD_df = {}
for city,tourist_pts in city_tourist_pt_dict.items():
    OD[city] = gn.calculate_OD(G, [city], tourist_pts, fail_value = 9999999)
    OD_df[city] = pd.DataFrame(OD[city], index = [city], columns = tourist_pts)

In [123]:
sample_df = OD_df[4784044133]
sample_df

,6117715989,2085208123,2078574693,1146790064,5497589950,1709670615,7083442431,5596123398,3686420776,849219917,...,4342594760,3301905899,198553180,2408066657,3615147754,1713880855,5497589553,2057039735,2057039753,77682630
4784044133,246.884367,10032.738931,5795.916461,5595.063132,2950.074792,7434.484289,487.724105,108.423476,1637.208436,1904.107684,...,3726.12794,5532.948146,432.959124,1479.318821,8967.031034,5902.910295,2800.297799,5480.234495,5469.468574,25.618408


## Now work on generating routes and visualizing them

In [ ]:
to node
4784044133
from node
3819995230

In [151]:
gn.example_edge(G,2)

(1650104033, 6236632580, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f64f9e97e10>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 9.757789924575977, 'speed': 26.793650793650794, 'time': 1.3110585040840237, 'mode': 'drive'})
(1650104033, 970058024, {'osm_id': 152177791, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f64f9e9c110>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 16.0020254611234, 'speed': 26.793650793650794, 'time': 2.150035174515869, 'mode': 'drive'})


In [159]:
G.edges([1650104033,6236632580])

OutMultiEdgeDataView([(1650104033, 6236632580), (1650104033, 970058024), (6236632580, 6236632579), (6236632580, 1650104033)])

In [160]:
G.edges([1650104033, 970058024])

OutMultiEdgeDataView([(1650104033, 6236632580), (1650104033, 970058024), (970058024, 1650104033), (970058024, 970058027), (970058024, 33389196)])

In [161]:
G.edges([4784044133,3819995230])

OutMultiEdgeDataView([(4784044133, 2035763030), (4784044133, 3819995230), (3819995230, 4784044133), (3819995230, 4776878438)])

In [162]:
G.get_edge_data(4784044133, 3819995230)

{0: {'osm_id': 378589933,
  'geometry': <shapely.geometry.linestring.LineString at 0x7f64b26f5a90>,
  'infra_type': 'secondary',
  'min_speed': nan,
  'max_speed': nan,
  'mean_speed': nan,
  'length': 1.21376041538949,
  'speed': 50,
  'time': 0.08739074990804328,
  'mode': 'drive'}}

In [167]:
G.get_edge_data(4784044133, 3819995230)[0]['geometry'].coords

In [141]:
from shapely.ops import linemerge
#linemerge(lines)

In [192]:
def tabulate_edges(route):
    edge_table = []
    route_geometry = LineString()
    for idx in range(0, len(route) - 1):
        edge_table.append([route[idx], route[idx+1]])
        # look up line
        #print('to node')
        #print(route[idx])
        #print('from node')
        #print(route[idx+1])
        edge_geometry = G.get_edge_data(route[idx],route[idx+1])[0]['geometry']
        route_geometry = route_geometry.union(edge_geometry)
    print('print route_geometry')
    print(route_geometry)
    
    return(edge_table, route_geometry)

In [193]:
#complete_routes = gpd.GeoDataFrame(columns=['o', 'd', 'geometry'])

In [194]:
#complete_routes

,o,d,geometry


In [216]:
from itertools import islice
LIMIT = 1

    
complete_edges = []
complete_routes = []


count = 0

# This will iterate through all of the values in the OD Matrix, therefore you will see edge being counted 
# in both directions, for example if there is an edge of node 1 to node 2, then there will be an edge 
# corresponding to of node 2 to node 1

# for origin, row in sample_df.iterrows(): 
for origin, row in islice(sample_df.iterrows(), LIMIT):    
    for destination, value in islice(row.items(), LIMIT):
        try:
            origin = int(origin)
        except:
            pass
        try:
            destination = int(destination)
        except:
            pass
        
        count = count + 1
        
        route = nx.dijkstra_path(G, origin, destination, weight = 'time')
        #path_edges = zip(route,route[1:])
        #print('print path_edges')
        #print(list(path_edges))
        # print(route)
        edge_table, route_geometry = tabulate_edges(route)
        print('print edge_table:')
        print(edge_table)
        complete_edges = complete_edges + edge_table
        #print('route_time')
        #print(value)
        complete_routes.append([edge_table[0][0], edge_table[-1][1], value, route_geometry])
        print('edge_table[:-1]')
        print(edge_table[-1][1])

print route_geometry
MULTILINESTRING ((80.7670837 6.9738216, 80.76707500000001 6.9738283), (80.7670928 6.9738167, 80.7670837 6.9738216), (80.7670928 6.9738167, 80.7670457 6.973783), (80.7670457 6.973783, 80.7670078 6.9737515), (80.7670078 6.9737515, 80.7669736 6.9737191), (80.7669736 6.9737191, 80.7669571 6.9736939), (80.7669571 6.9736939, 80.7669453 6.9736601), (80.7669453 6.9736601, 80.76693400000001 6.973613), (80.76693400000001 6.973613, 80.7669259 6.9735561), (80.7669259 6.9735561, 80.7669059 6.9733364), (80.7669059 6.9733364, 80.76686189999999 6.9727634), (80.766161 6.9732913, 80.766249 6.9730985, 80.7663088 6.9730028, 80.7663728 6.9729042, 80.7664632 6.9727654, 80.7664728 6.9727505, 80.7664824 6.9727385, 80.7664946 6.9727306, 80.7665136 6.9727272, 80.7665373 6.9727275, 80.76686189999999 6.9727634), (80.7660403 6.9732288, 80.766161 6.9732913), (80.7659816 6.9731976, 80.7660403 6.9732288), (80.7659352 6.9731663, 80.7659816 6.9731976), (80.7658029 6.9730515, 80.7659352 6.9731663), 

In [217]:
complete_routes

[[4784044133,
  6117715989,
  246.88436721196808,

In [218]:
complete_routes_df = pd.DataFrame(complete_routes, columns=['origin','destination','time','geometry'])

In [219]:
complete_routes_df

,origin,destination,time,geometry
0,4784044133,6117715989,246.884367,"(LINESTRING (80.7670837 6.9738216, 80.76707500..."


In [212]:
complete_routes_gdf = gpd.GeoDataFrame(complete_routes_df, crs='epsg:4326')

In [213]:
complete_routes_gdf

,origin,destination,geometry
0,4784044133,6117715989,"MULTILINESTRING ((80.76708 6.97382, 80.76708 6..."


In [214]:
# export to shapefile

In [172]:
complete_edges

[[4784044133, 3819995230],
 [3819995230, 4776878438],
 [4776878438, 6720549511],
 [6720549511, 6720549512],
 [6720549512, 4790267482],
 [4790267482, 6720549510],
 [6720549510, 6720549504],
 [6720549504, 6720549509],
 [6720549509, 4776878436],
 [4776878436, 191850645],
 [191850645, 191850698],
 [191850698, 247162948],
 [247162948, 4775747483],
 [4775747483, 4775747484],
 [4775747484, 247162945],
 [247162945, 4775747485],
 [4775747485, 336184347],
 [336184347, 77682693],
 [77682693, 2829005549],
 [2829005549, 6117715995],
 [6117715995, 6117715994],
 [6117715994, 6117715999],
 [6117715999, 6117715993],
 [6117715993, 6117715992],
 [6117715992, 6117715998],
 [6117715998, 6117715991],
 [6117715991, 6117715990],
 [6117715990, 6117715997],
 [6117715997, 6117715989]]

In [173]:
complete = pd.DataFrame(complete_edges, columns = ['o', 'd'])
complete['w'] = 1
complete_count = pd.DataFrame(complete.groupby(['o','d']).count())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [131]:
#complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on = 'o', right_on='node_id')
#complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on = 'd', right_on='node_id')
#complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w']]

In [132]:
complete_count

,o,d,w
0,49364661,5018474153,1
1,49364685,5018474148,1
2,49364700,5018474164,1
3,49364719,4571633797,1
4,49364756,5018474144,1
...,...,...,...
21096,7095172437,49369047,18
21097,7229317182,3385797191,2
21098,7279591542,598167787,5
21099,7323225814,598167898,5


### testing 

In [82]:
OD_df[OD_df > 20000]

,2291208197,1667080206,6991570964,6117715989,1743126554,2100457519,1185558575,130445367,2085208123,4047450172,...,3772792789,3772792790,1730404311,258523099,6564888541,3216934896,1506164725,1657419767,1675585534,4621961215
4140985760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21992.550175,NaN,...,NaN,NaN,20191.061674,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4784044133,NaN,NaN,NaN,NaN,NaN,26436.198542,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24446.349165,NaN,NaN,NaN,NaN,NaN
1734550181,22253.893418,NaN,NaN,NaN,NaN,28808.955754,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,20767.059336,26819.106377,NaN,NaN,NaN,22026.702570,NaN
4004620073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3379679501,23632.568454,NaN,NaN,NaN,NaN,27835.477944,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,25845.628567,NaN,NaN,NaN,23405.377606,NaN
3289453904,20834.449315,NaN,NaN,NaN,NaN,21754.640756,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20772.276043,NaN
1243386867,NaN,21028.939802,NaN,21176.982762,NaN,NaN,20301.939039,22674.661147,25899.034561,NaN,...,20230.809405,20220.354595,24473.158962,NaN,NaN,NaN,23580.780562,NaN,NaN,NaN
7098642100,23877.890534,28003.185631,26374.805251,29229.796481,23656.088094,NaN,27276.184869,29648.906977,34941.923384,21650.517516,...,26965.507630,26955.052820,33516.047784,NaN,NaN,25167.001175,30555.026391,NaN,23815.717262,26318.535591
1727494683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5385270047,22760.369817,NaN,NaN,NaN,NaN,23551.239017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,21561.389640,NaN,NaN,NaN,22698.196545,NaN


In [83]:
# 9999999
OD_df[OD_df > 20000].count()

2291208197    5
1667080206    2
6991570964    1
6117715989    2
1743126554    1
             ..
3216934896    1
1506164725    2
1657419767    0
1675585534    5
4621961215    1
Length: 337, dtype: int64

In [92]:
pd.Series.ge(OD_df[OD_df > 9999998].count(), 2).value_counts()

False    337
dtype: int64

## test airports

In [67]:
airports = gpd.read_file('./origins_destinations/intl_airport.shp')

In [68]:
airports

,osm_id,emergencyh,aeroway,operatorty,source,capacitype,addrfull,emergency,name,addrcity,building,POINT_X,POINT_Y,geometry
0,4.197465e+09,None,helipad,None,None,None,None,None,Ratmalana Int'l Airport,None,None,79.885533,6.823861,POINT (79.88553 6.82386)
1,1.912607e+08,None,None,None,None,None,None,None,Mattala Rajapaksa Int'l Airport,None,None,0.000000,0.000000,POINT (81.12338 6.28615)
2,7.479128e+08,None,None,None,None,None,None,None,Jaffna Int'l Airport,None,None,0.000000,0.000000,POINT (80.06853 9.79439)


In [69]:
airports_gdf = gn.pandana_snap_c(G, airports, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [70]:
airports_list = list(set(airports_gdf.NN))

In [71]:
airport_OD = gn.calculate_OD(G, airports_list, destinations_list, fail_value = 9999999)

In [72]:
airport_OD_df = pd.DataFrame(airport_OD, index = airports_list, columns = destinations_list)

In [73]:
airport_OD_df

,2291208197,1667080206,6991570964,6117715989,1743126554,2100457519,1185558575,130445367,2085208123,4047450172,...,3772792789,3772792790,1730404311,258523099,6564888541,3216934896,1506164725,1657419767,1675585534,4621961215
4075647849,24033.093498,28158.388596,26530.008215,29384.999445,23811.291059,9480.562118,27431.387833,29804.109941,35097.126348,21805.720480,...,27120.710595,27110.255784,33671.250749,17873.240241,5591.199142,25322.204140,30710.229356,12620.810646,23970.920226,26473.738556
5456722972,23962.943130,2729.046989,1582.915207,16657.734032,11238.089139,24993.359934,401.104665,5274.042297,14162.375077,11981.972575,...,670.014065,680.468876,11312.086677,16951.463517,23003.510557,7961.552420,6180.161711,15418.675025,23900.769858,1738.544752
3261840927,18637.304574,8269.627751,10611.644448,12086.846317,17745.441512,31500.712308,9320.546367,6776.161683,10405.112134,18073.755040,...,9847.748914,9858.203725,1991.552584,18571.012278,29510.862931,14193.040409,5520.125862,21926.027399,18410.113726,10798.849008
